In [1]:
import os, sys

#Allow relative imports to directories above lesson1. adds another '..' to the sys.path list
sys.path.insert(1, os.path.join(sys.path[0], '..'))
print(sys.path)

['', '..', '/home/cole/anaconda2/lib/python27.zip', '/home/cole/anaconda2/lib/python2.7', '/home/cole/anaconda2/lib/python2.7/plat-linux2', '/home/cole/anaconda2/lib/python2.7/lib-tk', '/home/cole/anaconda2/lib/python2.7/lib-old', '/home/cole/anaconda2/lib/python2.7/lib-dynload', '/home/cole/.local/lib/python2.7/site-packages', '/home/cole/anaconda2/lib/python2.7/site-packages', '/home/cole/anaconda2/lib/python2.7/site-packages/PIL', '/home/cole/anaconda2/lib/python2.7/site-packages/Sphinx-1.5.6-py2.7.egg', '/home/cole/anaconda2/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg', '/home/cole/anaconda2/lib/python2.7/site-packages/IPython/extensions', '/home/cole/.ipython']


In [2]:
# Import modules
import os, sys
from utils import *
from vgg16 import Vgg16
from shutil import copyfile

WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
Using TensorFlow backend.


In [3]:
data_dir = '/home/cole/FastAI/Data/Dogs vs Cats/'

In [4]:
#Instantiate plotting tool
#In Jupyter notebooks, you will need to run this command before doing any plotting
%matplotlib inline

## Action Plan
1. Create Validation and Sample sets
2. Rearrange image files into their respective directories 
3. Finetune and Train model
4. Generate predictions
5. Validate predictions
6. Submit predictions to Kaggle

# Create a validation set and sample !!!*DR START*!!!

In [6]:
%cd $data_dir
%mkdir valid
%mkdir results
%mkdir -p sample/train
%mkdir -p sample/test
%mkdir -p sample/valid
%mkdir -p sample/results
%mkdir -p test/unknown

/home/cole/FastAI/Data/Dogs vs Cats


In [7]:
cd $data_dir/train

/home/cole/FastAI/Data/Dogs vs Cats/train


In [9]:
# Finds all files that hava a similar pattern and stores it in a list. Aka all files that end with .jpg.
g = glob('*.jpg')

In [10]:
# Shuffles glob list and stores a copy in a value (shuf).
shuf = np.random.permutation(g)

['cat.2855.jpg' 'dog.7006.jpg' 'dog.10768.jpg' ..., 'cat.5026.jpg' 'cat.7382.jpg' 'cat.6925.jpg']


In [11]:
# Move 1 percent of Train files to Data's valid
for i in range(int(len(g) * 0.01)):
     os.rename(shuf[i], data_dir + 'valid/' + shuf[i])

In [13]:
# Copy 200 of Train files to Sample's train
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(200):
    copyfile(shuf[i], data_dir + 'sample/train/' + shuf[i])

24750


In [14]:
# Copy 50 of Trains valid files to Sample valid
%cd $data_dir/valid

g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(50): 
    copyfile(shuf[i], data_dir + 'sample/valid/' + shuf[i])

/home/cole/FastAI/Data/Dogs vs Cats/valid


## Rearange files into respective subdirectories

In [15]:
%cd $data_dir/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $data_dir/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $data_dir/sample/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $data_dir/sample/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $data_dir/test
%mv *.jpg unknown/

/home/cole/FastAI/Data/Dogs vs Cats/train
/home/cole/FastAI/Data/Dogs vs Cats/valid
/home/cole/FastAI/Data/Dogs vs Cats/sample/train
/home/cole/FastAI/Data/Dogs vs Cats/sample/valid
/home/cole/FastAI/Data/Dogs vs Cats/test


## Finetune and train !!!*DR END*!!!

In [6]:
%cd $data_dir

path = data_dir + 'sample/'
test_dir = data_dir + 'test/'
results_dir = data_dir + 'results/'
train_dir = path + 'train/'
valid_dir = path + 'valid/'

/home/cole/FastAI/Data/Dogs vs Cats


In [7]:
# Import Vgg helper class
vgg = Vgg16()

In [8]:
# Set constants. Can be experimented with.
batch_size = 64
epoch_no = 3

In [ ]:
# Finetune the model
batches = vgg.get_batches(train_dir, batch_size = batch_size)
val_batches = vgg.get_batches(valid_dir, batch_size = batch_size * 2)
vgg.finetune(batches)

vgg.model.optimizer.lr = 0.01

Found 200 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [ ]:
latest_weights_filename = None

for epoch in range(epoch_no):
    print("Running epoch number: ", epoch)
    vgg.fit(batches, val_batches, nb_epoch=1)
    latest_weights_filename = "ft%d.h5" % epoch
    vgg.model.save_weights(result_dir + latest_weights_filename)
print("Completed %s of fit operations" % epoch_no)

('Running epoch number: ', 0)
Epoch 1/1


## Generating predictions

In [ ]:
batches, preds = vgg.test(test_dir, batch_size = batch_size * 2)

Found 12500 images belonging to 1 classes.


In [ ]:
# For every image, vgg.test generates 2 probabilites, based on how we ordered our folders
print(preds(:5))

filenames = batches.filenames
print(filenames(:5))

In [ ]:
# You can verify the column ordering by viewing images
from PIL import image

Image.open(test_dir + filenames[2])

In [4]:
# Save array for later use
save_array(results_dir + 'test_preds.dat', preds)
save_array(results_dir + 'filenames_dat', filenames)

NameError: name 'save_array' is not defined

### Looking at the overall metrics, it's also a good idea to look at examples of each of:
1. A few correct labels at random
2. A few incorrect labels at random
3. The most correct labels of each class (ie those with highest probability that are correct)
4. The most incorrect labels of each class (ie those with highest probability that are incorrect)
5. The most uncertain labels (ie those with probability closest to 0.5).

In [5]:
# Calculate predictions on validation set
vgg.model.load_weights(results_dir + latest_weights_filename)

NameError: name 'vgg' is not defined

In [7]:
val_batches, probs = vgg.test(valid_dir, batches_size = batches_size)

NameError: name 'vgg' is not defined

In [ ]:
filenames = val_batches.filenames
expected_labels = val_batches.classes #0 or 1

# Round our probs to assign labels
our_preds = preds[:,0]
our_labels = np.round(1-our_preds)


In [3]:
from keras.preprocessing import image

# Helper function to print images by index
def plot_idx(idx, titles = None):
    plot([image.load_image(valid_dir + filenames[i])], titles = titles)
    
# Number of images to view for each vizualization 
view_no = 4

In [2]:
#1. A few correct labels at random

correct = np.where(our_lables == expected_labels)[0]
print("Found %d correct labels" % len(correct))
idx = permutation(correct)[:view_no]
plots_idx(idx, our_predictions[idx])

NameError: name 'np' is not defined

In [4]:
#2. A few incorrect labels at random

incorrect = np.where(our_labels != expecte_labels)[0]
print("Foud %d incorrect labels" % len(incorrect))
idx = permutation(incorrect)[:view_no]
plot_idx(idx, our_predicitons[idx])


NameError: name 'np' is not defined

In [5]:
#3. Images we were most confident were cats and were actually cats

correct_cats = np.where((our_labels == 0) & (our_labels == expected_labels))[0]
print("Found %d confident cat labels that were cats" % len(correct_cats))
most_correct_cats = np.argsort(our_predictions[correct_cats])[::-1][:view_no]
plot_idx(correct_cats[most_correct_cats], our_predictions[correct_cats][most_correct_cats])

NameError: name 'np' is not defined

In [7]:
#4. Images we were most confident were dogs and were actually dogs

correct_dogs = np.where((our_labels == 1) & (our_labels == expected_labels))[0]
print("Found %d confident cat labels that were cats" % len(correct_dogs))
most_correct_dogs = np.argsort(our_predictions[correct_cats])[:view_no]
plots_idx(correct_dogs[most_correct_dogs], our_predictions[correct_dogs][most_correct_dogs])


NameError: name 'np' is not defined

In [9]:
#5. Images we were most confident were cats but are actually dogs
incorrect_cats = np.where((our_labels == 0) & (our_labels != expected_labels))[0]
print("Found %d false cats " % len(incorrect_cats))
if len(incorrect_cats):
    most_incorrect = np.argsort(our_predictions[incorrect_cats])[::-1][:view_no]
    plots_idx(incorrect_cats[most_incorrect], our_predictions[incorrect_cats][most_incorrect_cats])

NameError: name 'np' is not defined

In [1]:
#6. Images we were most confident were dogs but are actually cats
incorrect_dogs = np.where((our_labels == 1) & (our_labels != expected_labels))[0]
print("Found %d false dogs " % len(incorrect_dogs))
if len(incorrect_dogs):
        most_incorrect_dogs = np.argsort(our_predictions[incorrect_dogs])[:view_no]
        plot_idx(incorrect_dogs[most_incorrect_dogs], our_predicitons[incorrect_dogs][most_incorrect_dogs])

NameError: name 'np' is not defined

In [2]:
#7. The most uncertain labels
most_uncertain = np.argsort(np.abs(our_predictions - 0.5))
plot_idx(most_uncertain[:view_no], our_predicitons[most_uncertain])

NameError: name 'np' is not defined

In [1]:
from sklearn import confusion_matrix
cm = confusion_matrix(expected_labels, our_labels)

ImportError: cannot import name confusion_matrix

In [2]:
plot_confusion_matrix(cm, val_batches.class_indices)

NameError: name 'plot_confusion_matrix' is not defined

## Submit

In [4]:
# Load our test predictions from file
preds = load_array(results_dir + 'test_preds.dat')
filenames = load_array(results_dir + 'filenames.dat')

NameError: name 'load_array' is not defined

In [5]:
# Grab the dog prediciton column
isdog = preds[:,1]
print("Raw Predictions: " + str(isdog[:5]))
print("Mid Predicitons: " + str(isdog[(isdog < 0.6) & (isdog > 0.4)]))
print("Edge Predictions: ") + str(isdog[(isdog == 1) | (isdog == 0)])

NameError: name 'preds' is not defined

In [8]:
# Visualize log-loss when true value = 1
# Y-axis is log-loss. X-axis is the probability that label = 1
# Log-los approaches infinity as we approch 0% probability

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import log_loss

x = [i*.0001 for i in range(1,10000)]

y = [log_loss([1],[[i* 0.0001,1-(i*.0001)]],eps=1e-14) for i in range(1,10000,1
                                                                      
plt.plot(x, y)
plt.axis([-.05, 1.1, -.8, 10])
plt.title("Log Loss when true label = 1")
plt.xlabel("predicted probability")
plt.ylabel("log loss")

plt.show()

IndentationError: expected an indented block (<ipython-input-8-44b76ed8f41b>, line 12)

In [9]:
# Round down our edge prediciton so all 1's are 0.95 and all zeros are 0.05
isdog = isdog.clip(min = 0.05, max = 0.95)


NameError: name 'isdog' is not defined

In [10]:
# Extract image ID's from our test/unknown dir.
filenames = batches.filenames
ids = np.array([int(f[8:f.find('.')]) for i in filenames])

NameError: name 'batches' is not defined

In [11]:
# Join the two columns into an array of [imageid, dog]
subm = np.stack([ids, isdog], axis = 1)
subm[:5]

NameError: name 'ids' is not defined

In [12]:
%cd data_dir
submission_file_name = 'submission1.csv'
np.savetext(submission_file_name, subm, fmt = '%d, %.5f', header = 'ID label', comments = '')


[Errno 2] No such file or directory: 'data_dir'
/home/cole/FastAI/My Notebooks


AttributeError: 'module' object has no attribute 'savetext'

In [14]:
from IPython.display import FileLink
FileLink(submission_file_name)

/home/cole/FastAI/My Notebooks/submission1.csv